In [ ]:
import cv2
import numpy as np

# Load the pre-trained MobileNet SSD model
model_path = r'C:\Users\Sparx\Downloads\object_detection_COCO-main\object_detection_COCO-main\frozen_inference_graph.pb'
config_path = r'C:\Users\Sparx\Downloads\object_detection_COCO-main\object_detection_COCO-main\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
net = cv2.dnn.readNetFromTensorflow(model_path, config_path)

# Initialize the video stream
cap = cv2.VideoCapture(0)  # Use 0 for built-in webcam, 1 for external webcam

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Prepare the frame to be fed into the model
    blob = cv2.dnn.blobFromImage(frame, size=(300, 300), swapRB=True)
    net.setInput(blob)
    
    # Run the object detection
    detections = net.forward()
    
    # Loop over the detections
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Filter out weak detections by confidence
            class_id = int(detections[0, 0, i, 1])
            
            # Draw bounding box for the detected object
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (startX, startY, endX, endY) = box.astype('int')
            
            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
            label = f'Class: {class_id}'
            cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('Object Detection using MobileNet SSD', frame)
    
    # Break the loop on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()
